# Building a Spam Filter with Naive Bayes
**Introduction** - In this project, we're going to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. Our goal is to write a program that classifies new messages with an accuracy greater than 80% — so we expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam).

The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). You can also download the dataset directly from [this link](https://dq-content.s3.amazonaws.com/433/SMSSpamCollection). The data collection process is described in more details on [this page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where you can also find some of the authors' papers.

**Goal** - Our goal is to create a spam filter that classifies new messages with an accuracy greater than 80%.

## Exploring Data set
Lets import libraries, read the data and see what it looks like.

In [1]:
import pandas as pd

sms_spam = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])
sms_spam.shape

(5572, 2)

In [2]:
sms_spam.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
sms_spam.tail()

,Label,SMS
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...
5571,ham,Rofl. Its true to its name


In [4]:
sms_spam['Label'].value_counts(normalize=True) * 100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

From the above exploration, we can see that the data set has 5572 rows and 2 columns. The SMS are classified in 'label' column as spam and ham(not spam).

We will be dividing thw whole data in training set (80% of data) to train the Naive Bayes and test set(20% of the data) for checking how well our algorithm works
## Training and test set
We will now divide the data as mentioned above. First lets randomize the data.

In [5]:
sms_spam = sms_spam.sample(frac=1, random_state = 1)

eighty_percent_index = round(len(sms_spam) * 0.8)
training_set = sms_spam.iloc[:eighty_percent_index]
test_set = sms_spam.iloc[eighty_percent_index:]

print("Training Set size: ", len(training_set))
print("\nTest Set size: ", len(test_set))
print("\nTest + Training set size: ", len(training_set) + len(test_set))

Training Set size:  4458

Test Set size:  1114

Test + Training set size:  5572


The splitting of data set is done. Lets reset the index labels for data data sets, since they will be having index labels from before the split.

In [6]:
training_set = training_set.reset_index()
test_set = test_set.reset_index()

Now that we have reset the index labels, lets check if the proportion of spam and ham messages is the same for both the sets.

In [7]:
# Training Set
print("Training set frequency")
training_set['Label'].value_counts(normalize=True) * 100

Training set frequency


ham     86.54105
spam    13.45895
Name: Label, dtype: float64

In [8]:
# Test Set
print("Test set frequency")
test_set['Label'].value_counts(normalize=True) * 100

Test set frequency


ham     86.804309
spam    13.195691
Name: Label, dtype: float64

In our original data set, we had 86.59% spam and 13.40% ham messages. Looking at the above results. Hence, we can say that the percentages similar to what we have in the full dataset. So, this is a good representative of the original data set, and we are on track.

## Data Cleaning
To calculate all these probabilities, we'll first need to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need.

Let's start by removing punctuation from the set.

In [9]:
print("Before cleaning")
training_set.head()

Before cleaning


,index,Label,SMS
0,1078,ham,"Yep, by the pretty sculpture"
1,4028,ham,"Yes, princess. Are you going to make me moan?"
2,958,ham,Welp apparently he retired
3,4642,ham,Havent.
4,4674,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [10]:
# Removing punctuation
training_set['SMS'] = training_set['SMS'].str.replace('\W', ' ')
#  Making all letters lowercase
training_set['SMS'] = training_set['SMS'].str.lower()

print("After cleaning")
training_set.head()

After cleaning


,index,Label,SMS
0,1078,ham,yep by the pretty sculpture
1,4028,ham,yes princess are you going to make me moan
2,958,ham,welp apparently he retired
3,4642,ham,havent
4,4674,ham,i forgot 2 ask ü all smth there s a card on ...


## Creating Vocabulary

Now that we have finished cleaning the data, we will find the unique works in the full training set. This will be used as vocabulary later to calculate probabilities for Naive-Bayes. 

In [11]:
vocabulary = []

training_set['SMS'] = training_set['SMS'].str.split()

for message in training_set['SMS']:
    for word in message:
        vocabulary.append(word)
        
# Convering list to set to remove duplicates
vocabulary = set(vocabulary)
# Converting back to list
vocabulary = list(vocabulary)
#vocabulary.remove('index')

# First 10 examples
vocabulary[:10]

['07090201529',
 'outstanding',
 'shelf',
 'running',
 'talent',
 'studio',
 'nanny',
 'sight',
 'skillgame',
 'mcfly']

We have our list of uniques words now, which we can use for calculating probability of the SMS being spam based on each word of a message.

We will be creating a new DataFrame for this, in which most of the columns would be unique words, and the number of times they appeared in a SMS. However, we'll first build a dictionary that we'll then convert to the DataFrame we need.

In [12]:
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1         

The above code created a new dictionary and stores the number of times each word appeared in a message.

Now that we have the dictionary we need, let's do the final transformations to our training set and then move forward with creating the spam filter.

In [13]:
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


Let's concatenate our dataset with the original training set.

In [14]:
training_set_clean = pd.concat([training_set, word_counts], axis=1)
training_set_clean = training_set_clean.drop('index', axis = 1)
training_set_clean.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


Now that we're done with data cleaning and have a training set to work with, we can begin creating the spam filter. 

## Calculating Constants
Recall that the Naive Bayes algorithm will need to know the probability values of the two equations below to be able to classify new messages:

![P(Spam)](https://render.githubusercontent.com/render/math?math=P%28Spam%20%7C%20w_1%2Cw_2%2C%20...%2C%20w_n%29%20%5Cpropto%20P%28Spam%29%20%5Ccdot%20%5Cprod_%7Bi%3D1%7D%5E%7Bn%7DP%28w_i%7CSpam%29&mode=display)
![P(Ham)](https://render.githubusercontent.com/render/math?math=P%28Ham%20%7C%20w_1%2Cw_2%2C%20...%2C%20w_n%29%20%5Cpropto%20P%28Ham%29%20%5Ccdot%20%5Cprod_%7Bi%3D1%7D%5E%7Bn%7DP%28w_i%7CHam%29&mode=display)

While the values for every probability in the product will be;
![Pwi(Spam)](https://render.githubusercontent.com/render/math?math=P%28w_i%7CSpam%29%20%3D%20%5Cfrac%7BN_%7Bw_i%7CSpam%7D%20%2B%20%5Calpha%7D%7BN_%7BSpam%7D%20%2B%20%5Calpha%20%5Ccdot%20N_%7BVocabulary%7D%7D&mode=display)
![Pwi(Ham)](https://render.githubusercontent.com/render/math?math=P%28w_i%7CHam%29%20%3D%20%5Cfrac%7BN_%7Bw_i%7CHam%7D%20%2B%20%5Calpha%7D%7BN_%7BHam%7D%20%2B%20%5Calpha%20%5Ccdot%20N_%7BVocabulary%7D%7D&mode=display)

Some of the terms in the four equations above will have the same value for every new message. We can calculate the value of these terms once and avoid doing the computations again when a new messages comes in. Below, we'll use our training set to calculate:
  - P(Spam) and P(Ham)
  - NSpam, NHam, NVocabulary
We'll also use Laplace smoothing and set $\alpha = 1$.

In [15]:
p_spam = training_set['Label'].value_counts()['spam'] / len(training_set)
p_ham = training_set['Label'].value_counts()['ham'] / len(training_set)
print("P(spam)", p_spam)
print("P(ham)", p_ham)

P(spam) 0.13458950201884254
P(ham) 0.8654104979811574


In [16]:
spam_messages = training_set_clean[training_set_clean['Label'] == 'spam']
ham_messages = training_set_clean[training_set_clean['Label'] == 'ham']
vocabulary.remove('index')

# N_Spam
n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()

# N_Ham
n_words_per_ham_message = ham_messages['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()

# N_Vocabulary
n_vocabulary = len(vocabulary)

# Laplacian smoothing
alpha = 1

In the above snippet, we have calculated sum of number of words in spam messages as well as ham messages. We also calculated the number of unique words in the data set, and set the laplacian smoothing parameter.

As we've already mentioned, all these terms will have constant values in our equations for every new message (regardless of the message or each individual word in the message).

## Calculating parameters
We'll need to calculate P("secret"|Spam) for both these messages, and we can use the training set to get the values.
This means that we can use our training set to calculate the probability for each word in our vocabulary.

We have 7,783 words in our vocabulary, which means we'll need to calculate a total of 15,566 probabilities. For each word, we need to calculate both P(wi|Spam) and P(wi|Ham).

In more technical language, the probability values that P(wi|Spam) and P(wi|Ham) will take are called parameters.Let's now calculate all the parameters using the equations below:
![P(wi|Spam)](https://render.githubusercontent.com/render/math?math=P%28w_i%7CSpam%29%20%3D%20%5Cfrac%7BN_%7Bw_i%7CSpam%7D%20%2B%20%5Calpha%7D%7BN_%7BSpam%7D%20%2B%20%5Calpha%20%5Ccdot%20N_%7BVocabulary%7D%7D&mode=display)
![P(wi|Ham)](https://render.githubusercontent.com/render/math?math=P%28w_i%7CHam%29%20%3D%20%5Cfrac%7BN_%7Bw_i%7CHam%7D%20%2B%20%5Calpha%7D%7BN_%7BHam%7D%20%2B%20%5Calpha%20%5Ccdot%20N_%7BVocabulary%7D%7D&mode=display)

In [17]:
# Initializing parameter dictionaries
spam_parameters = {unique_word:0 for unique_word in vocabulary}
ham_parameters = {unique_word:0 for unique_word in vocabulary}

# Calculating probabilities for each word
for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum()
    n_word_given_ham = ham_messages[word].sum()
    
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    
    spam_parameters[word] = p_word_given_spam
    ham_parameters[word] = p_word_given_ham


Now that we've calculated all the constants and parameters we need, we can start creating the spam filter. The spam filter can be understood as a function that:
  - Takes in as input a new message (w1, w2, ..., wn)
  - Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
  - Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
    - If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
    - If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
    - If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.
    
Also, to write the code for calculating probability it is spam  given the message and probability ham given the message, we need to use these two equations:
![P(Spam) | message](https://render.githubusercontent.com/render/math?math=P%28Spam%20%7C%20w_1%2Cw_2%2C%20...%2C%20w_n%29%20%5Cpropto%20P%28Spam%29%20%5Ccdot%20%5Cprod_%7Bi%3D1%7D%5E%7Bn%7DP%28w_i%7CSpam%29&mode=display)
![P(Ham) | message](https://render.githubusercontent.com/render/math?math=P%28Ham%20%7C%20w_1%2Cw_2%2C%20...%2C%20w_n%29%20%5Cpropto%20P%28Ham%29%20%5Ccdot%20%5Cprod_%7Bi%3D1%7D%5E%7Bn%7DP%28w_i%7CHam%29&mode=display)

Now we'll write the code for calculating p_spam_given_message and p_ham_given_message, and then we'll use the function to classify two new messages. On the next screen, we'll classify all the 1,114 messages in our test set.

In [18]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham  
    
    for word in message:
        if word in spam_parameters:
            p_spam_given_message *= spam_parameters[word]
        if word in ham_parameters:
            p_ham_given_message *= ham_parameters[word]
        
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

Now that we have written the function, let's test it by using a few examples.

In [19]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3486572848464234e-25
P(Ham|message): 1.9370730139733197e-27
Label: Spam


In [20]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4379803356617713e-25
P(Ham|message): 3.6879274559428634e-21
Label: Ham


The classifier works! Let's try it on our test set.

## Measuring the Spam Filter's Accuracy
The two results above look promising, but let's see how well the filter does on our test set, which has 1,114 messages.

We'll start by writing a function that returns classification labels instead of printing them.

In [21]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in spam_parameters:
            p_spam_given_message *= spam_parameters[word]

        if word in ham_parameters:
            p_ham_given_message *= ham_parameters[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

We have the function. Let's use it, and put the result for each message in a new column of the test set.

In [22]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()

,index,Label,SMS,predicted
0,2131,ham,Later i guess. I needa do mcat study too.,ham
1,3418,ham,But i haf enuff space got like 4 mb...,ham
2,3424,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,1538,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,5393,ham,"All done, all handed in. Don't know if mega sh...",ham


Now, we'll write a function to measure the accuracy of our spam filter to find out how well our spam filter does.

In [23]:
correct = 0
total = len(test_set)

for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1

print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total * 100, "%")

Correct: 1100
Incorrect: 14
Accuracy: 98.74326750448833 %


The accuracy is 98.74% which is really great. 
Our spam filter looked at 1,114 messages that it hasn't seen in training, and classified 1,100 correctly.